<h1>Desarrollo de API tomando datos de solr a kibana</h1>


<h3>Conexion e indexacion a kibana</h3>

In [12]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ConnectionError
import logging

# Establecer nivel de registro a DEBUG
#logging.basicConfig(level=logging.DEBUG)
try:
    # Especifique la URL de la API de Elastic
    es_url = "http://34.145.100.55/:9200"


    # Especifique la ruta del archivo .crt sde Elastic
    es_crt = "/home/chaki/Documentos/PROYECTOS/Woombat/TRABAJO/API.kibana/kibana-test.crt"

    #usuario para acceder a elastisearch
    usr = "elastic"
    contr = "elastic"

    # Cree una instancia de Elasticsearch, pasando la URL y la conexión SSL con el archivo .crt
    es = Elasticsearch(['https://34.105.28.82:9200'],
                       verify_certs=False,
                       ca_certs=es_crt,
                       basic_auth=(usr,contr))
    
    # Evalua conexion al servidor
    if es.ping():
        print('Conexión exitosa!')
    else:
        print('No se pudo conectar.')

    # Creamos un documento a indexar
    documento = {
        'titulo': 'Mi primer documento indexado',
        'texto': 'Este es el contenido de mi primer documento en Elasticsearch'
    }

    res = es.index(index='test-api-index-1', id=1, document=documento)
    es.indices.create_data_stream()

    
except ConnectionError as error:
    print("Error al conectar:", error)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): 34.105.28.82:9200
/home/chaki/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host '34.105.28.82'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
DEBUG:urllib3.connectionpool:https://34.105.28.82:9200 "HEAD / HTTP/1.1" 200 0
INFO:elastic_transport.transport:HEAD https://34.105.28.82:9200/ [status:200 duration:0.407s]
/home/chaki/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host '34.105.28.82'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Conexión exitosa!


DEBUG:urllib3.connectionpool:https://34.105.28.82:9200 "PUT /test-api-index-1/_doc/1 HTTP/1.1" 201 149
INFO:elastic_transport.transport:PUT https://34.105.28.82:9200/test-api-index-1/_doc/1 [status:201 duration:0.320s]


<h3>Validar la creacion del indice haciendo un request</h3>


In [ ]:
from elasticsearch import Elasticsearch

# Especifique la URL de la API de Elastic
es_url = "http://34.145.100.55/:9200"

#usuario para acceder a elastisearch
usr = "elastic"
contr = "elastic"

# Cree una instancia de Elasticsearch, pasando la URL y las credenciales de autenticación
es = Elasticsearch(['https://34.105.28.82:9200'],
                       verify_certs=False,
                       ca_certs=es_crt,
                       basic_auth=(usr,contr))

# Escriba una consulta para recuperar todos los documentos en el índice 'mi-indice'
query = {
    "query": {
        "match_all": {}
    }
}

# Realice la consulta utilizando el método search de la instancia de Elasticsearch
result = es.search(index='test-api-index', body=query)

# Imprima los resultados
for hit in result['hits']['hits']:
    print(hit['_source'])


<h3>API para extraer datos de Solr</h3>

In [ ]:
import requests
import json
import logging


logging.basicConfig(level=logging.DEBUG)
# URL de la consulta a Solr
url = "http://35.203.142.113:8983/solr/soat_test_1/select?fl=servicio_consumido%3Aoperacion_realizada%2Cperiodo%2C%20nro_solicitud%3Anro_id_sesion%2C%20fecha_operacion%3Afecha_hora_operacion%2C%20hora_operacion%3Afecha_hora_operacion%2C%20fecha_aprobacion%3Afecha_hora_operacion%2C%20hora_aprobacion%3Afecha_hora_operacion%2C%20fecha_creacion_solicitud%3Afecha_hora_operacion%2C%20hora_creacion_solicitud%3Afecha_hora_operacion%2C%20fecha_otp%3Afecha_hora_operacion%2C%20hora_otp%3Afecha_hora_operacion%2C%20fecha_emision%3Afecha_hora_operacion%2C%20hora_emision%3Afecha_hora_operacion%2C%20cod_agente_vendedor%2C%20cod_oficina%2C%20nro_otp%3Atoken_otp_ent%2C%20direccion_ip%3Adireccion_ip%2C%20tipo_identificacion_cliente%3Atipo_identificacion%2C%20nro_identificacion_cliente%3Anro_identificacion%2C%20canal_comunicacion%3Acod_canal%2C%20flg_cliente_propietario%3Atipo_identificacion%2C%20cod_motivo_rechazo_ingreso%3Acaracter_aceptacion%2C%20etapa_proceso%3Acaracter_aceptacion%2C%20nombres_cliente%3Aprimer_nombre_cliente_sal%2C%20primer_apellido_cliente%3Aprimer_apellido_cliente_sal%2C%20segundo_apellido_cliente%3Asegundo_apellido_cliente_sal%2C%20nro_celular_cliente%3Anro_celular_principal_sal%2C%20origen_recursos%3Acod_tipo_recaudo_ent%2C%20nro_producto_origen_recursos_enm%3Anro_producto_ent%2C%20nro_producto_origen_recursos%3Anro_producto_ent%2C%20cant_cuotas%3Acant_cuotas_ent%2C%20tipo_identificacion_propietario%3Atipo_identificacion%2C%20nro_identificacion_propietario%3Anro_identificacion%2C%20nombres_propietario%3Anombres_sal%2C%20apellidos_propietario%3Aapellidos_sal%2C%20nro_celular_propietario%3Anro_celular_sal%2C%20direccion_propietario%3Adireccion_sal%2C%20cod_ciudad_propietario%3Acod_ciudad_sal%2C%20correo_electronico_entrega%3Acorreo_electronico_sal%2C%20nro_cotizacion%3Anro_poliza_sal%2C%20placa_vehiculo%3Aplaca_ent%2C%20marca_vehiculo%3Amarca_vehiculo_sal%2C%20modelo_vehiculo%3Amodelo_sal%2C%20fecha_inicio_vigencia%3Afecha_inicio_sal%2C%20cilindraje_vehiculo%3Acilindraje_sal%2C%20linea_vehiculo%3Alinea_sal%2C%20monto_soat%3A%27valor_prima_sal%2Bvalor_fosyga_sal%2Bvalor_runt_sal%27%2C%20valor_soat_prima%3Avalor_prima_sal%2C%20valor_soat_fosyga%3Avalor_fosyga_sal%2C%20valor_soat_runt%3Avalor_runt_sal%2C%20cod_motivo_no_emision_soat%3Acaracter_aceptacion%2C%20terminos_condiciones_aceptados%3Atipo_documento&group=false%20collapse%3Dnro_id_sesion&indent=true&q.op=OR&q=nro_id_sesion%3A%220002b2d0-ddc7-11ec-945b-275be0706b3a%22"

# Realizar la solicitud HTTP a Solr
response = requests.get(url)
data = json.loads(response.content)


# Verificar que la solicitud fue exitosa (código de estado HTTP 200)
if response.status_code == 200:

     # Acceder al contenido del campo "response"
    response_data = data["response"]

    # Imprimir el contenido del campo "response"
    

    print("Datos guardados correctamente en respuestaSolr.json.\estado respuesta del servidor:", response.status_code)
    print(response_data)
else:
    print("La solicitud a Solr falló con el siguiente código de estado HTTP:", response.status_code)


<h3>conexion a la API de kibana</h3>


In [ ]:
import requests
from elasticsearch import Elasticsearch

index_name = ''

try:
    mapping = es.indices.get_mapping(index=index_name)
    fields = mapping[index_name]['mappings']['properties'].keys()
except Exception as e:
    print(f'Error al obtener el mapping: {e}')
    fields = []

try:
    kibana_url = ''
    headers = {'kbn-xsrf': 'true'}
    payload = {
        'attributes': {
            'title': index_name+"_data_view",
            'timeFieldName': 'timestamp',
            'description': '',
            'fields': ','.join(fields),
            'kibanaSavedObjectMeta': {
                'searchSourceJSON': {
                    'index': index_name,
                    'query': {
                        'language': 'kuery',
                        'query': ''
                    },
                    'filter': []
                }
            }
        }
    }
    response = requests.post(
        f'{kibana_url}/api/saved_objects/data-view',
        headers=headers,
        json=payload
    )
    if response.status_code == 200:
        print(f'Data View creado exitosamente: {response.json()}')
    else:
        print(f'Error al crear el Data View: {response.text}')
except Exception as e:
    print(f'Error al crear el Data View: {e}')



<h3>Merge de los codigos de solr con kibana</h3>


In [ ]:
import requests
import json
from elasticsearch import Elasticsearch

# URL de la consulta a Solr
solr_url = "http://34.145.7.234:8983/solr/soat_test_1/select?fl=servicio_consumido%3Aoperacion_realizada%2Cperiodo%2C%20nro_solicitud%3Anro_id_sesion%2C%20fecha_operacion%3Afecha_hora_operacion%2C%20hora_operacion%3Afecha_hora_operacion%2C%20fecha_aprobacion%3Afecha_hora_operacion%2C%20hora_aprobacion%3Afecha_hora_operacion%2C%20fecha_creacion_solicitud%3Afecha_hora_operacion%2C%20hora_creacion_solicitud%3Afecha_hora_operacion%2C%20fecha_otp%3Afecha_hora_operacion%2C%20hora_otp%3Afecha_hora_operacion%2C%20fecha_emision%3Afecha_hora_operacion%2C%20hora_emision%3Afecha_hora_operacion%2C%20cod_agente_vendedor%2C%20cod_oficina%2C%20nro_otp%3Atoken_otp_ent%2C%20direccion_ip%3Adireccion_ip%2C%20tipo_identificacion_cliente%3Atipo_identificacion%2C%20nro_identificacion_cliente%3Anro_identificacion%2C%20canal_comunicacion%3Acod_canal%2C%20flg_cliente_propietario%3Atipo_identificacion%2C%20cod_motivo_rechazo_ingreso%3Acaracter_aceptacion%2C%20etapa_proceso%3Acaracter_aceptacion%2C%20nombres_cliente%3Aprimer_nombre_cliente_sal%2C%20primer_apellido_cliente%3Aprimer_apellido_cliente_sal%2C%20segundo_apellido_cliente%3Asegundo_apellido_cliente_sal%2C%20nro_celular_cliente%3Anro_celular_principal_sal%2C%20origen_recursos%3Acod_tipo_recaudo_ent%2C%20nro_producto_origen_recursos_enm%3Anro_producto_ent%2C%20nro_producto_origen_recursos%3Anro_producto_ent%2C%20cant_cuotas%3Acant_cuotas_ent%2C%20tipo_identificacion_propietario%3Atipo_identificacion%2C%20nro_identificacion_propietario%3Anro_identificacion%2C%20nombres_propietario%3Anombres_sal%2C%20apellidos_propietario%3Aapellidos_sal%2C%20nro_celular_propietario%3Anro_celular_sal%2C%20direccion_propietario%3Adireccion_sal%2C%20cod_ciudad_propietario%3Acod_ciudad_sal%2C%20correo_electronico_entrega%3Acorreo_electronico_sal%2C%20nro_cotizacion%3Anro_poliza_sal%2C%20placa_vehiculo%3Aplaca_ent%2C%20marca_vehiculo%3Amarca_vehiculo_sal%2C%20modelo_vehiculo%3Amodelo_sal%2C%20fecha_inicio_vigencia%3Afecha_inicio_sal%2C%20cilindraje_vehiculo%3Acilindraje_sal%2C%20linea_vehiculo%3Alinea_sal%2C%20monto_soat%3A%27valor_prima_sal%2Bvalor_fosyga_sal%2Bvalor_runt_sal%27%2C%20valor_soat_prima%3Avalor_prima_sal%2C%20valor_soat_fosyga%3Avalor_fosyga_sal%2C%20valor_soat_runt%3Avalor_runt_sal%2C%20cod_motivo_no_emision_soat%3Acaracter_aceptacion%2C%20terminos_condiciones_aceptados%3Atipo_documento&group=false%20collapse%3Dnro_id_sesion&indent=true&q.op=OR&q=nro_id_sesion%3A%220002b2d0-ddc7-11ec-945b-275be0706b3a%22"

# Realizar la solicitud HTTP a Solr
solr_response = requests.get(solr_url)
solr_data = json.loads(solr_response.content)

# Verificar que la solicitud fue exitosa (código de estado HTTP 200)
if solr_response.status_code == 200:
    # Acceder al contenido del campo "response"
    response_data = solr_data["response"]

    # Especifique la URL de la API de Elastic
    es_url = "http://34.82.188.82:9200"

    # Especifique la ruta del archivo .crt sde Elastic
    es_crt = "/home/chaki/Documentos/PROYECTOS/Woombat/TRABAJO/API.kibana/kibana-test.crt"
    
    # usuario para acceder a Elasticsearch
    usr = "elastic"
    contr = "elastic"

    # Cree una instancia de Elasticsearch, pasando la URL y la conexión SSL con el archivo .crt
    es = Elasticsearch(['https://34.82.188.82:9200'],
                       verify_certs=False,
                       ca_certs=es_crt,
                       basic_auth=(usr,contr))

    # Evalua conexion al servidor
    if es.ping():
        print('Conexión exitosa!')
    else:
        print('No se pudo conectar.')

    # Indexar los documentos en Elasticsearch
    for doc in response_data["docs"]:
        es.index(index='test-api-index-2', body=doc)
    

else:
    print("La solicitud a Solr falló con el siguiente código de estado HTTP:", solr_response.status_code)


In [ ]:
import requests
import json
from elasticsearch import Elasticsearch
from pyspark.sql import SparkSession

# Función para conectar a Solr y obtener los datos
def get_solr_data(solr_url):
    solr_response = requests.get(solr_url)
    solr_data = json.loads(solr_response.content)
    if solr_response.status_code == 200:
        print("La solicitud a Solr fue un EXITO con el siguiente código de estado HTTP:", solr_response.status_code)
        response_data = solr_data["response"]
        return response_data["docs"]
    else:
        print("La solicitud a Solr FALLÓ con el siguiente código de estado HTTP:", solr_response.status_code)
        return None

# Función para conectar a Elasticsearch
def connect_to_es(es_url, es_crt, usr, contr):
    es = Elasticsearch(
        [es_url],
        http_auth=(usr, contr),
        verify_certs=False,
        ca_certs=es_crt
    )
    if es.ping():
        print('Conexión a elastic exitosa!')
        return es
    else:
        print('No se pudo conectar a elastic.')
        return None

# Función para enviar los datos de Solr a Elasticsearch
def send_data_to_es(es, index, data):
    for doc in data:
        es.index(index=index, body=doc)

# Funcion para crear un dataview en kibana en base al indice de elasticsearch
def create_dataview(ki_url, es, index):
    # Se define el nombre del índice
    index_name = index

    # Se intenta obtener el mapeo (mapping) del índice desde Elasticsearch
    try:
        mapping = es.indices.get_mapping(index=index_name)
        # Si se logra obtener el mapping, se extraen los campos del índice
        fields = mapping[index_name]['mappings']['properties'].keys()
    except Exception as e:
        # En caso de error al obtener el mapping, se asigna una lista vacía a fields
        print(f'Error al obtener el mapping: {e}')
        fields = []

    # Se intenta crear el Data View en Kibana
    try:
        kibana_url = ki_url
        headers = {'kbn-xsrf': 'true'}
        payload = {
            'attributes': {
                # Se define el título del Data View, concatenando el nombre del índice
                'title': index_name+"_data_view",
                # Se especifica el campo de tiempo del índice
                'timeFieldName': 'timestamp',
                'description': '',
                # Se especifican los campos del índice para ser incluidos en el Data View
                'fields': ','.join(fields),
                'kibanaSavedObjectMeta': {
                    'searchSourceJSON': {
                        # Se especifica el índice a ser utilizado en la consulta
                        'index': index_name,
                        'query': {
                            'language': 'kuery',
                            'query': ''
                        },
                        'filter': []
                    }
                }
            }
        }
        # Se envía la solicitud POST para crear el Data View
        response = requests.post(
            f'{kibana_url}/api/saved_objects/data-view',
            headers=headers,
            json=payload
        )
        # Si el código de estado de la respuesta es 200, el Data View fue creado con éxito
        if response.status_code == 200:
            print(f'Data View creado exitosamente: {response.status_code}')
        else:
            # De lo contrario, se imprime el código de estado como mensaje de error
            print(f'Error al crear el Data View: {response.status_code}')
    except Exception as e:
        # Si ocurre cualquier otro error durante la creación del Data View, se imprime el mensaje de error
        print(f'Error al crear el Data View: {e}')


# Obtener los datos de Solr
solr_url = ""
docs = get_solr_data(solr_url)

# Conectar a Elasticsearch
es_url = "https://34.145.100.55:9200"
es_crt = "/home/chaki/Documentos/PROYECTOS/Woombat/TRABAJO/API.kibana/kibana-test.crt"
usr = "elastic"
contr = "elastic"
es = connect_to_es(es_url, es_crt, usr, contr)

# Enviar los datos de Solr a Elasticsearch
if docs and es:
    index = "test-api-index-2"
    send_data_to_es(es, index, docs)

    # procesamiento de datos con PySpark antes de enviarlos a Elasticsearch
    spark = SparkSession.builder.appName("SolrToES").getOrCreate()
    rdd = spark.sparkContext.parallelize(docs)
    rdd_filtered = rdd.filter(lambda x: x["score"] > 0.5)
    data_filtered = rdd_filtered.collect()
    send_data_to_es(es, index, data_filtered)
    spark.stop()

<h3>Codigo de manera funcional implementado pyspark</h3>

In [ ]:
import requests
import json
from elasticsearch import Elasticsearch
from pyspark.sql import SparkSession

# Función para conectar a Solr y obtener los datos
def get_solr_data(solr_url):
    solr_response = requests.get(solr_url)
    solr_data = json.loads(solr_response.content)
    if solr_response.status_code == 200:
        print("La solicitud a Solr fue un EXITO con el siguiente código de estado HTTP:", solr_response.status_code)
        response_data = solr_data["response"]
        return response_data["docs"]
    else:
        print("La solicitud a Solr FALLÓ con el siguiente código de estado HTTP:", solr_response.status_code)
        return None

# Función para conectar a Elasticsearch
def connect_to_es(es_url, es_crt, usr, contr):
    es = Elasticsearch(
        [es_url],
        http_auth=(usr, contr),
        verify_certs=False,
        ca_certs=es_crt
    )
    if es.ping():
        print('Conexión a elastic exitosa!')
        return es
    else:
        print('No se pudo conectar a elastic.')
        return None

# Función para enviar los datos de Solr a Elasticsearch
def send_data_to_es(es, index, data):
    for doc in data:
        es.index(index=index, body=doc)


# Conectar a Elasticsearch
es_url = "https://34.145.100.55:9200"
es_crt = "/home/chaki/Documentos/PROYECTOS/Woombat/TRABAJO/API.kibana/kibana-test.crt"
usr = "elastic"
contr = "elastic"
es = connect_to_es(es_url, es_crt, usr, contr)

# Enviar los datos de Solr a Elasticsearch
if docs and es:
    index = "test-api-index-2"
    send_data_to_es(es, index, docs)

    # procesamiento de datos con PySpark antes de enviarlos a Elasticsearch
    spark = SparkSession.builder.appName("SolrToES").getOrCreate()
    rdd = spark.sparkContext.parallelize(docs)
    rdd_filtered = rdd.filter(lambda x: x["score"] > 0.5)
    data_filtered = rdd_filtered.collect()
    send_data_to_es(es, index, data_filtered)
    spark.stop()


<h3>Version sin pyspark</h3>

In [24]:
import requests
import json
from elasticsearch import Elasticsearch
from pyspark.sql import SparkSession

# Función para conectar a Solr y obtener los datos
def get_solr_data(solr_url):
    solr_response = requests.get(solr_url)
    solr_data = json.loads(solr_response.content)
    if solr_response.status_code == 200:
        print("La solicitud a Solr fue un EXITO con el siguiente código de estado HTTP:", solr_response.status_code)
        response_data = solr_data["response"]
        return response_data["docs"]
    else:
        print("La solicitud a Solr FALLÓ con el siguiente código de estado HTTP:", solr_response.status_code)
        return None

# Función para conectar a Elasticsearch
def connect_to_es(es_url, es_crt, usr, contr):
    es = Elasticsearch(
        [es_url],
        http_auth=(usr, contr),
        verify_certs=False,
        ca_certs=es_crt
    )
    if es.ping():
        print('Conexión a elastic exitosa!')
        return es
    else:
        print('No se pudo conectar a elastic.')
        return None

# Función para enviar los datos de Solr a Elasticsearch
def send_data_to_es(es, index, data):
    for doc in data:
        es.index(index=index, body=doc)

# Funcion para crear un dataview en kibana en base al indice de elasticsearch
def create_dataview(ki_url, es, index, username, password):
    # Se define el nombre del índice
    index_name = index
    # Se intenta obtener el mapeo (mapping) del índice desde Elasticsearch
    try:
        mapping = es.indices.get_mapping(index=index_name)
        # Si se logra obtener el mapping, se extraen los campos del índice
        fields = mapping[index_name]['mappings']['properties'].keys()
    except Exception as e:
        # En caso de error al obtener el mapping, se asigna una lista vacía a fields
        print(f'Error al obtener el mapping: {e}')
        fields = []

    # Se intenta crear el Data View en Kibana
    try:
        kibana_url = ki_url
        headers = {
                    'kbn-xsrf': 'true',
                    'Authorization':'Basic {base64.b64encode('+username+':'+password+').decode()}'
                   }
        payload = {
            'attributes': {
                # Se define el título del Data View, concatenando el nombre del índice
                'title': index_name+"_data_view",
                # Se especifica el campo de tiempo del índice
                'timeFieldName': 'timestamp',
                'description': '',
                # Se especifican los campos del índice para ser incluidos en el Data View
                'fields': ','.join(fields),
                'kibanaSavedObjectMeta': {
                    'searchSourceJSON': {
                        # Se especifica el índice a ser utilizado en la consulta
                        'index': index_name,
                        'query': {
                            'language': 'kuery',
                            'query': ''
                        },
                        'filter': []
                    }
                }
            }
        }
        # Se envía la solicitud POST para crear el Data View
        response = requests.post(
            f'{kibana_url}/api/saved_objects/'+index_name+'_data_view',
            headers=headers,
            json=payload
        )
        # Si el código de estado de la respuesta es 200, el Data View fue creado con éxito
        if response.status_code == 200:
            print(f'Data View creado exitosamente: {response.status_code}')
        else:
            # De lo contrario, se imprime el código de estado como mensaje de error
            print(f'Error al crear el Data View: {response.status_code}')
    except Exception as e:
        # Si ocurre cualquier otro error durante la creación del Data View, se imprime el mensaje de error
        print(f'Error al crear el Data View: {e}')

def main():
    es_url = "https://34.82.188.82:9200"
    kiba_url = "http://34.82.188.82:5601"
    # Obtener los datos de Solr
    Consulta_solr = "http://34.145.7.234:8983/solr/soat_test_1/select?fl=servicio_consumido%3Aoperacion_realizada%2Cperiodo%2C%20nro_solicitud%3Anro_id_sesion%2C%20fecha_operacion%3Afecha_hora_operacion%2C%20hora_operacion%3Afecha_hora_operacion%2C%20fecha_aprobacion%3Afecha_hora_operacion%2C%20hora_aprobacion%3Afecha_hora_operacion%2C%20fecha_creacion_solicitud%3Afecha_hora_operacion%2C%20hora_creacion_solicitud%3Afecha_hora_operacion%2C%20fecha_otp%3Afecha_hora_operacion%2C%20hora_otp%3Afecha_hora_operacion%2C%20fecha_emision%3Afecha_hora_operacion%2C%20hora_emision%3Afecha_hora_operacion%2C%20cod_agente_vendedor%2C%20cod_oficina%2C%20nro_otp%3Atoken_otp_ent%2C%20direccion_ip%3Adireccion_ip%2C%20tipo_identificacion_cliente%3Atipo_identificacion%2C%20nro_identificacion_cliente%3Anro_identificacion%2C%20canal_comunicacion%3Acod_canal%2C%20flg_cliente_propietario%3Atipo_identificacion%2C%20cod_motivo_rechazo_ingreso%3Acaracter_aceptacion%2C%20etapa_proceso%3Acaracter_aceptacion%2C%20nombres_cliente%3Aprimer_nombre_cliente_sal%2C%20primer_apellido_cliente%3Aprimer_apellido_cliente_sal%2C%20segundo_apellido_cliente%3Asegundo_apellido_cliente_sal%2C%20nro_celular_cliente%3Anro_celular_principal_sal%2C%20origen_recursos%3Acod_tipo_recaudo_ent%2C%20nro_producto_origen_recursos_enm%3Anro_producto_ent%2C%20nro_producto_origen_recursos%3Anro_producto_ent%2C%20cant_cuotas%3Acant_cuotas_ent%2C%20tipo_identificacion_propietario%3Atipo_identificacion%2C%20nro_identificacion_propietario%3Anro_identificacion%2C%20nombres_propietario%3Anombres_sal%2C%20apellidos_propietario%3Aapellidos_sal%2C%20nro_celular_propietario%3Anro_celular_sal%2C%20direccion_propietario%3Adireccion_sal%2C%20cod_ciudad_propietario%3Acod_ciudad_sal%2C%20correo_electronico_entrega%3Acorreo_electronico_sal%2C%20nro_cotizacion%3Anro_poliza_sal%2C%20placa_vehiculo%3Aplaca_ent%2C%20marca_vehiculo%3Amarca_vehiculo_sal%2C%20modelo_vehiculo%3Amodelo_sal%2C%20fecha_inicio_vigencia%3Afecha_inicio_sal%2C%20cilindraje_vehiculo%3Acilindraje_sal%2C%20linea_vehiculo%3Alinea_sal%2C%20monto_soat%3A%27valor_prima_sal%2Bvalor_fosyga_sal%2Bvalor_runt_sal%27%2C%20valor_soat_prima%3Avalor_prima_sal%2C%20valor_soat_fosyga%3Avalor_fosyga_sal%2C%20valor_soat_runt%3Avalor_runt_sal%2C%20cod_motivo_no_emision_soat%3Acaracter_aceptacion%2C%20terminos_condiciones_aceptados%3Atipo_documento&group=false%20collapse%3Dnro_id_sesion&indent=true&q.op=OR&q=nro_id_sesion%3A%220002b2d0-ddc7-11ec-945b-275be0706b3a%22"
    docs = get_solr_data(Consulta_solr)
    # Conectar a Elasticsearch

    es_crt = "/home/chaki/Documentos/PROYECTOS/Woombat/TRABAJO/API.kibana/kibana-test.crt"
    usr = "elastic"
    contr = "elastic"
    # Crear conexion con Kibana
    es = connect_to_es(es_url, es_crt, usr, contr)
    index = "test-api-index-3"
    # Enviar data a kibana
    send_data_to_es(es, index, docs)
    # Crear data view para visuaizar datos en kibana
    create_dataview(kiba_url, es, index, usr, contr)
  
if __name__ == "__main__":
    main()

   



DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 34.145.7.234:8983
DEBUG:urllib3.connectionpool:http://34.145.7.234:8983 "GET /solr/soat_test_1/select?fl=servicio_consumido%3Aoperacion_realizada%2Cperiodo%2C%20nro_solicitud%3Anro_id_sesion%2C%20fecha_operacion%3Afecha_hora_operacion%2C%20hora_operacion%3Afecha_hora_operacion%2C%20fecha_aprobacion%3Afecha_hora_operacion%2C%20hora_aprobacion%3Afecha_hora_operacion%2C%20fecha_creacion_solicitud%3Afecha_hora_operacion%2C%20hora_creacion_solicitud%3Afecha_hora_operacion%2C%20fecha_otp%3Afecha_hora_operacion%2C%20hora_otp%3Afecha_hora_operacion%2C%20fecha_emision%3Afecha_hora_operacion%2C%20hora_emision%3Afecha_hora_operacion%2C%20cod_agente_vendedor%2C%20cod_oficina%2C%20nro_otp%3Atoken_otp_ent%2C%20direccion_ip%3Adireccion_ip%2C%20tipo_identificacion_cliente%3Atipo_identificacion%2C%20nro_identificacion_cliente%3Anro_identificacion%2C%20canal_comunicacion%3Acod_canal%2C%20flg_cliente_propietario%3Atipo_identificacion%2C%20cod

La solicitud a Solr fue un EXITO con el siguiente código de estado HTTP: 200


/home/chaki/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host '34.82.188.82'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
DEBUG:urllib3.connectionpool:https://34.82.188.82:9200 "HEAD / HTTP/1.1" 200 0
INFO:elastic_transport.transport:HEAD https://34.82.188.82:9200/ [status:200 duration:0.521s]
/tmp/ipykernel_21111/2593355160.py:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=index, body=doc)
/home/chaki/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host '34.82.188.82'. Adding certificate verification is strongly advised. See: 

Conexión a elastic exitosa!


DEBUG:urllib3.connectionpool:https://34.82.188.82:9200 "POST /test-api-index-3/_doc HTTP/1.1" 201 168
INFO:elastic_transport.transport:POST https://34.82.188.82:9200/test-api-index-3/_doc [status:201 duration:0.875s]
/home/chaki/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host '34.82.188.82'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
DEBUG:urllib3.connectionpool:https://34.82.188.82:9200 "POST /test-api-index-3/_doc HTTP/1.1" 201 168
INFO:elastic_transport.transport:POST https://34.82.188.82:9200/test-api-index-3/_doc [status:201 duration:0.142s]
/home/chaki/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host '34.82.188.82'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs

Error al crear el Data View: 401
